In [1]:
%%javascript
/** Below because this notebook is pretty large */
var rto = 200;
console.log('NB: Increase require timeout to ' + rto + ' seconds');
window.requirejs.config({waitSeconds: rto});

<IPython.core.display.Javascript object>

## Importacion de Librerías y Configuración de Consola

In [2]:
import time 
import os
import janitor
import pandas as pd
import numpy as numpy
from pandas_datareader import data as pdr
from pathlib import Path
import pickle

# Antes de utilizar esta librería, descargarla. Ingresar en terminal: pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

In [3]:
print()

In [4]:
# Especificamos el directorio base, retrocediendo una carpeta
BASE = Path('..')
# Y, a partir de allí, definimos las carpetas que debieran existir
RAW = BASE/'raw'
NOTEBOOKS = BASE/'notebooks'
DATA = BASE/'data'
REFERENCIAS = BASE/'referencias'
PERFORMANCE = BASE/'performance'
MODELOS = BASE/'modelos'

In [5]:
# Creamos todos los directorios que definimos previamente, si no existieran aún.
path_list = [RAW, NOTEBOOKS, DATA, REFERENCIAS, PERFORMANCE]
for path_i in path_list:
    if not os.path.exists(path_i):
        os.makedirs(path_i)

In [6]:
#Tomo el tiempo al inicio de la ejecucion
start_time=time.time()

In [7]:
#Seteamos para que no utilice notacion cientifica
pd.options.display.float_format = '{:.9f}'.format
#Seteo para que el máximo de columnas que muestra al levantar una base sean 500
pd.set_option('display.max_columns',500)
#Estos códigos hacen que la visualización de la consola abarque toda la pantalla (sin los recortes a los costados). Tambien hacen que al mostrar dataframes podamos ver todas las columnas que tiene.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [8]:
# Codigo para poder imprimir multiples outputs en una misma línea
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
# # Definiremos el directorio en el que estamos trabajando. 
# scripts=os.getcwd()  #Obtenemos la direccion actual
# os.chdir('..') # Nos vamos una carpeta atrás
# raiz=os.getcwd() # Esta direccion la guardaremos como "raiz" 
# os.chdir(raiz+'\\raw') 
# raw=os.getcwd()
# os.chdir(raiz+'\\raw')
# outputs=os.getcwd()

# Importacion de Archivos

## Ingresamos Fecha de Hoy

In [10]:
from datetime import date
today = date.today()
Today_Date = str(today)

## Informacion BCBA

### Índices Merval

In [11]:
# Importamos el indice Merval
ticker_merval = '^MERV'
indice_merval = pdr.get_data_yahoo(ticker_merval, start="2002-01-11", end=Today_Date)
indice_merval['ticker'] = 'merval'
indice_merval.head()
indice_merval.tail(2)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2002-01-17,322.970001221,375.829986572,318.920013428,371.200012207,371.200012207,0,merval
2002-01-18,377.149993896,422.299987793,377.119995117,414.859985352,414.859985352,0,merval
2002-01-21,414.859985352,500.589996338,414.859985352,470.549987793,470.549987793,0,merval
2002-01-22,480.859985352,482.540008545,441.209991455,445.880004883,445.880004883,0,merval
2002-01-23,438.670013428,453.230010986,423.230010986,438.890014648,438.890014648,0,merval


,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2021-02-17,51466.210937500,52646.460937500,51466.210937500,52386.589843750,52386.589843750,0,merval
2021-02-18,52386.589843750,53731.738281250,51400.718750000,51826.910156250,51826.910156250,0,merval


In [12]:
# Reseteamos index, para que 'Date' esté entre las columnas
df_merval = indice_merval.reset_index()
# Limpiamos el nombre de las columnas (todo en minusculas, sin espacios ni caracteres raros)
df_merval = (janitor.clean_names(df_merval))
# Quitamos la columna 'close'
df_merval.drop(columns='close', inplace=True)
# Y nombramos como 'close' a la que se llamaba 'adj_close' (ajustada por yahoo, esta es la posta)
df_merval.rename(columns={'adj_close':'close'}, inplace=True)
# mostramos
df_merval.head(2)

,date,open,high,low,close,volume,ticker
0,2002-01-17,322.970001221,375.829986572,318.920013428,371.200012207,0,merval
1,2002-01-18,377.149993896,422.299987793,377.119995117,414.859985352,0,merval


In [13]:
df_merval.tail()

,date,open,high,low,close,volume,ticker
4662,2021-02-10,52266.750000000,52653.289062500,51930.949218750,51940.910156250,0,merval
4663,2021-02-11,51940.910156250,52429.980468750,51563.488281250,51654.558593750,0,merval
4664,2021-02-12,51654.558593750,52104.648437500,51245.578125000,51466.210937500,0,merval
4665,2021-02-17,51466.210937500,52646.460937500,51466.210937500,52386.589843750,0,merval
4666,2021-02-18,52386.589843750,53731.738281250,51400.718750000,51826.910156250,0,merval


### Acciones Argentinas

In [14]:
# Importamos las cotizaciones históricas, hasta hoy, de las acciones argentinas desde Yahoo Finance
# Ojo: A veces la informacion está mal. La librería funciona bien, scrapea la pagina correctamente.
# pero es la página de Yahoo la que le pifia (ej: BBAR muestra que todos los dias la cotizacion fue 142.5, cualquiera)
lista_tickers_acciones_argentinas = ['auso',
                                     'agro',
                                     'ggal',
                                     'bhip',
                                     'bma',
                                     'bpat',
                                     'brio',
                                     'byma',
                                     'ctio',
#                                      'cvh',
                                     'edn',
                                     'ferr',
                                     'fipl',
                                     'gcla',
                                     'hava',
#                                      'bbar',
                                     'grim',
                                     'lede',
                                     'long',
                                     'metr',
                                     'moli',
                                     'mori',
#                                      'mvia',
                                     'supv',
                                     'ypfd'
                                     ]
lista_tickers_acciones_argentinas_original = lista_tickers_acciones_argentinas.copy()
# df_acciones_arg = pdr.get_data_yahoo(lista_tickers_acciones_argentinas[0] + ".BA", start="2002-01-11", end=Today_Date)[['Adj Close']]
# df_acciones_arg.columns=[lista_tickers_acciones_argentinas[0]]
# del lista_tickers_acciones_argentinas[0]

# Creamos dict vacio para luego colocarle la data
dict_acciones_argentinas = {}

for ticker in lista_tickers_acciones_argentinas:
    # download dataframe using pandas_datareader
    df_aux = pdr.get_data_yahoo(ticker + ".BA", start="2002-01-11", end=Today_Date)
    # Agregamos una columna especificando el ticker
    df_aux['ticker'] = ticker
    # quitamos la columna Close
    df_aux.drop(columns='Close', inplace=True)
    # reseteamos el index, así cuando limpiemos las columnas, 'date' estará incluida
    df_aux.reset_index(inplace=True)
    # limpiamos el nombre de todas las columnas
    df_aux = janitor.clean_names(df_aux)
    # reemplazamos adj_close por el nombre 'close', que es a lo que refiere
    df_aux.rename(columns={'adj_close':'close'}, inplace=True)
    # al dataframe le limpiamos los nombres de las columnas con janitor, y lo copiamos dentro del dict
    dict_acciones_argentinas[ticker] = df_aux.copy()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

# Serie Dolar Blue

## Importamos desde una página las cotizaciones desde 2015 hasta hoy

In [15]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

def scrap(año, mes):
    url  = 'https://www.cotizacion-dolar.com.ar/dolar-blue-historico-'+str(año)+'.php'
    for i in range(1,7):
        try:        
            fecha = datetime.datetime(año,mes,i)
            data = {'fecha': fecha.strftime('%d-%m-%y')}
            resp = requests.post(url, data=data)
            soup = BeautifulSoup(resp.text, "html.parser")
            break
        except:
            print('Falló en ',i)    
    filas = soup.find_all('td', {'style' : 'padding: 1%'})
    return filas

def parsear(filas):
    mensual = pd.DataFrame() 
    for i in range(1, int(len(list(filas))/3)):
        dic = {}
        dic['fecha'] = filas[3*i].text
        dic['bid'] = filas[3*i+1].text
        dic['ask'] = filas[3*i+2].text
        rueda = pd.DataFrame.from_dict(dic, orient='index').transpose().set_index('fecha')
        rueda.index = pd.to_datetime(rueda.index, format='%d-%m-%y ')
        mensual = pd.concat([mensual,rueda], axis=0)
    return mensual

def downloadAño(año):
    tablaAnual = pd.DataFrame()
    for i in range(1,13):
        filas = scrap(año=año, mes=i)
        tabla = parsear(filas)
        tablaAnual = pd.concat([tablaAnual,tabla],axis=0)
        print('mes',i, 'del año ', str(año), 'listo')        
#     tablaAnual.to_excel('blue_'+str(año)+'.xlsx')
    return tablaAnual 
    print(tablaAnual)


In [16]:
dolar_blue = {}
for i in np.arange(2015, 2022, 1):  
    dolar_blue[str(i)] = downloadAño(i)

mes 1 del año  2015 listo
mes 2 del año  2015 listo
mes 3 del año  2015 listo
mes 4 del año  2015 listo
mes 5 del año  2015 listo
mes 6 del año  2015 listo
mes 7 del año  2015 listo
mes 8 del año  2015 listo
mes 9 del año  2015 listo
mes 10 del año  2015 listo
mes 11 del año  2015 listo
mes 12 del año  2015 listo
mes 1 del año  2016 listo
mes 2 del año  2016 listo
mes 3 del año  2016 listo
mes 4 del año  2016 listo
mes 5 del año  2016 listo
mes 6 del año  2016 listo
mes 7 del año  2016 listo
mes 8 del año  2016 listo
mes 9 del año  2016 listo
mes 10 del año  2016 listo
mes 11 del año  2016 listo
mes 12 del año  2016 listo
mes 1 del año  2017 listo
mes 2 del año  2017 listo
mes 3 del año  2017 listo
mes 4 del año  2017 listo
mes 5 del año  2017 listo
mes 6 del año  2017 listo
mes 7 del año  2017 listo
mes 8 del año  2017 listo
mes 9 del año  2017 listo
mes 10 del año  2017 listo
mes 11 del año  2017 listo
mes 12 del año  2017 listo
mes 1 del año  2018 listo
mes 2 del año  2018 listo
mes

In [17]:
df_db = pd.concat(dolar_blue.values(), ignore_index=False)
df_db.head(2)
df_db.tail(2)

,bid,ask
fecha,,
2015-07-22,$ 14.80,$ 14.90
2015-07-23,$ 14.51,$ 14.66


,bid,ask
fecha,,
2021-02-18,$ 144.40,$ 148.00
2021-02-19,$ 144.40,$ 148.00


## Desde un excel traemos los datos más antiguos

In [18]:
db_ambito = pd.read_excel(RAW/'serie_dolar_blue.xlsx' )
db_ambito = db_ambito[['Fecha','Venta']]
db_ambito.head(2)
db_ambito.tail(2)

,Fecha,Venta
0,2020-10-01,"147,00"
1,2020-09-30,"146,00"


,Fecha,Venta
4648,2002-01-14,"1,68"
4649,2002-01-11,"1,70"


## Generamos la serie histórica completa de dolar blue

In [19]:
# Reseteamos el index de la primer serie, por comodidad
df_db.reset_index(inplace=True)
# Cruzamos los dataframes
cruza_db = pd.merge(db_ambito, df_db, how='outer', left_on='Fecha', right_on='fecha')
# Vemos cual es la fecha que corresponde. Si en un lado es null, usamos la del otro, y así
cruza_db['Fecha_OK'] = np.where(cruza_db['Fecha'].isnull(), cruza_db['fecha'], cruza_db['Fecha'])
# ordenamos todo por esta nueva fecha
cruza_db.sort_values(by='Fecha_OK',inplace=True)
# en el valor de venta reemplazamos la coma decimal por el punto (python usa sistema americano)
cruza_db['Venta'] = cruza_db['Venta'].str.replace(',','.')
# quitamos el símbolo de dinero para poder convertir 'ask' en numérica
cruza_db['ask'] = cruza_db['ask'].str.replace('$','')
# vemos cual es la diferencia entre las cotizaciones, pero solo para donde no haya nulls en ningun caso (si hay null colocamos diferencia 0)
cruza_db['dif'] = np.where(~(cruza_db['Venta'].isnull()) | (cruza_db['ask'].isnull()),
                           cruza_db['Venta'].astype(float) - cruza_db['ask'].astype(float),
                           0)
# nuestro valor final será el de Venta, pero cuando sea null colocaremos el de 'ask'
cruza_db['Venta_DBlue'] = np.where(cruza_db['Venta'].isnull(), cruza_db['ask'], cruza_db['Venta']) 
# convertimos esa variable a float
cruza_db['Venta_DBlue'] = cruza_db['Venta_DBlue'].astype(float)
# nos quedamos solo con la fecha y la cotizacion finales
cruza_db = cruza_db[['Fecha_OK', 'Venta_DBlue']]
# las renombramos
cruza_db.columns=['date','venta_dblue']
# quitamos duplicados por fecha
cruza_db.drop_duplicates(subset='date', keep='first', inplace=True)
# mostramos
cruza_db

C:\Users\ruizf\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  if sys.path[0] == '':


,date,venta_dblue
4649,2002-01-11,1.700000000
4648,2002-01-14,1.680000000
4647,2002-01-15,1.950000000
4646,2002-01-16,1.870000000
4645,2002-01-17,1.970000000
...,...,...
4750,2021-02-15,150.000000000
4751,2021-02-16,150.000000000
4752,2021-02-17,149.000000000
4753,2021-02-18,148.000000000


# Exporta data

In [20]:
# Exportamos merval 
df_merval.to_pickle(DATA/'df_merval.pkl')
# Exportamos dolar blue
cruza_db.to_pickle(DATA/'df_dolarblue.pkl')
# Exportamos acciones
np.save(DATA/'dict_acciones_arg',dict_acciones_argentinas)

In [21]:
# Tomo el tiempo al final de la ejecucion
end_time=time.time()
print('El script demora', (end_time-start_time)/60, 'minutos')

El script demora 2.186114803949992 minutos
